## Algoritmos de optimización - Trabajo Práctico

# Nombre y Apellidos: Sergio Moya Rodrigo

## https://github.com/smoyarodrigo/Trabajo-Practico-Sergio_Moya 

## Problema:

Sesiones de doblaje

## Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las 
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio d 
grabación independientemente del número de tomas que se graben. No es posible grabar  s 
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por os 
servicios de los actores de doblaje sea el menor posible. Los datos son:

## Número de actores : 10
## Número de tomaas : 30
## Actores/Tomas : https://bit.ly/36D8IuK

## 1 indica que el actor participa en la toma 
## 0 en caso contrario

# Modelo
# - ¿Como represento el espacio de soluciones?
# - ¿Cual es la función objetivo?
# - ¿Como implemento las restricciones?

# Espacio de soluciones: Asignación de tomas a días en subconjuntos de máximo 6, asegurando compatibilidad de actores.

# Función objetivo: Minimizar los días utilizados.

# Restricciones: Cada toma se asigna a un día, máximo 6 tomas por día, y compatibilidad de actores en cada día.

# Visualizamos código: 


In [19]:
!pip install pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, PULP_CBC_CMD
import numpy as np

# Datos
num_tomas = 30
num_actores = 10
max_tomas_por_dia = 6

tomas_participacion = [
    [1,1,1,1,1,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0], [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0], [0,1,0,1,0,0,0,1,0,0], [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0], [1,1,0,0,0,1,0,0,0,0], [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0], [1,1,1,0,1,0,0,1,0,0], [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0], [1,0,1,0,0,1,0,0,0,0], [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1], [1,0,1,0,1,0,0,0,1,0], [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0], [1,0,0,0,1,1,0,0,0,0], [1,0,0,1,0,0,0,0,0,0]
]

# Máximo número de días posibles en el peor caso
num_dias_max = num_tomas  

dias = range(num_dias_max)
tomas = range(num_tomas)
actores = range(num_actores)

# Definir el problema
problema = LpProblem("Minimizar_Dias_Grabacion", LpMinimize)

# Variables de decisión
x = LpVariable.dicts("x", [(t, d) for t in tomas for d in dias], cat="Binary")
y = LpVariable.dicts("y", dias, cat="Binary")  # 1 si el día se usa

# Función objetivo: Minimizar número de días de grabación
problema += lpSum(y[d] for d in dias)

# Restricción 1: Cada toma debe asignarse exactamente a un día
for t in tomas:
    problema += lpSum(x[(t, d)] for d in dias) == 1

# Restricción 2: No más de 6 tomas por día
for d in dias:
    problema += lpSum(x[(t, d)] for t in tomas) <= max_tomas_por_dia * y[d]

# Restricción 3: Compatibilidad de actores
for d in dias:
    for a in actores:
        problema += lpSum(x[(t, d)] * tomas_participacion[t][a] for t in tomas) <= num_tomas * y[d]

# Resolver el problema
problema.solve(PULP_CBC_CMD(msg=False))

# Resultados
print(f"Días de grabación mínimos: {int(sum(y[d].varValue for d in dias))}")
for d in dias:
    if y[d].varValue == 1:
        tomas_dia = [t for t in tomas if x[(t, d)].varValue == 1]
        print(f"Día {d + 1}: Tomas {tomas_dia}")


Días de grabación mínimos: 5
Día 18: Tomas [1, 14, 15, 21, 27, 29]
Día 20: Tomas [3, 7, 8, 13, 16, 19]
Día 23: Tomas [0, 4, 11, 24, 25, 28]
Día 26: Tomas [5, 6, 10, 18, 22, 23]
Día 28: Tomas [2, 9, 12, 17, 20, 26]


## Análisis# 
¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

## El problema puede ser visto como una variación del problema de empaquetamiento binario (bin packing problem), donde:Las tomas son los elementos a empacar.Los días son los contenedores con una capacidad máxima de 6 tomas.Se deben cumplir restricciones adicionales de compatibilidad de actores.

## Número de particiones posibles:

# El número de maneras en que se pueden dividir 30. 30 tomas en bloques de hasta 6 es similar a una combinación restringida. Esto es exponencial en 30.

## El problema es exponencial en complejidad debido a la gran cantidad de formas de agrupar tomas y las restricciones de compatibilidad. Resolverlo exactamente con ILP puede ser computacionalmente costoso, por lo que algoritmos heurísticos como greedy o metaheurísticas (búsqueda local, algoritmos genéticos) pueden ser útiles.



## Diseño:

# Qué técnica utilizo y por qué

## Para resolver este problema, Para resolver este problema, la técnica que uso es Estrategia Heurística Greedy más adecuada es el uso de algoritmos de búsqueda con restricciones combinado con heurísticas de optimización. 

## Por qué Heurísticas?: Dado que el problema es NP-completo (similar al problema de coloreado de grafos), no es práctico buscar una solución exacta en un tiempo razonable Las heurísticas permiten encontrar soluciones aproximadas de buena calidad en un tiempo aceptable

In [26]:
import numpy as np

# Datos
num_tomas = 30
num_actores = 10
max_tomas_por_dia = 6

tomas_participacion = [
    [1,1,1,1,1,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0], [0,1,0,0,1,0,1,0,0,0],
    [1,1,0,0,0,0,1,1,0,0], [0,1,0,1,0,0,0,1,0,0], [1,1,0,1,1,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0], [1,1,0,0,0,1,0,0,0,0], [1,1,0,1,0,0,0,0,0,0],
    [1,1,0,0,0,1,0,0,1,0], [1,1,1,0,1,0,0,1,0,0], [1,1,1,1,0,1,0,0,0,0],
    [1,0,0,1,1,0,0,0,0,0], [1,0,1,0,0,1,0,0,0,0], [1,1,0,0,0,0,1,0,0,0],
    [0,0,0,1,0,0,0,0,0,1], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,0,1,0,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,0,0,0,0,1,0,1,0,0],
    [1,1,1,1,0,0,0,0,0,0], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
    [1,1,0,1,0,0,0,0,0,1], [1,0,1,0,1,0,0,0,1,0], [0,0,0,1,1,0,0,0,0,0],
    [1,0,0,1,0,0,0,0,0,0], [1,0,0,0,1,1,0,0,0,0], [1,0,0,1,0,0,0,0,0,0]
]

# Heurística Greedy para asignación de días
dias = []  # Lista de días, cada día contiene una lista de tomas

for t, toma in enumerate(tomas_participacion):
    asignado = False
    
    for dia in dias:
        # Verificar si cabe en el día (máx 6 tomas) y no hay conflicto de actores
        if len(dia) < max_tomas_por_dia:
            actores_ocupados = np.any(np.array([tomas_participacion[i] for i in dia]), axis=0)
            if not np.any(np.array(toma) & actores_ocupados):  # No hay conflictos
                dia.append(t)
                asignado = True
                break
    
    # Si no pudo asignarse, abrir un nuevo día
    if not asignado:
        dias.append([t])

# Resultados
print(f"Días de grabación necesarios: {len(dias)}")
for i, dia in enumerate(dias):
    print(f"Día {i+1}: Tomas {dia}")


Días de grabación necesarios: 22
Día 1: Tomas [0, 20]
Día 2: Tomas [1, 3]
Día 3: Tomas [2, 13, 15]
Día 4: Tomas [4, 16]
Día 5: Tomas [5, 17]
Día 6: Tomas [6, 23]
Día 7: Tomas [7, 26]
Día 8: Tomas [8]
Día 9: Tomas [9]
Día 10: Tomas [10]
Día 11: Tomas [11]
Día 12: Tomas [12]
Día 13: Tomas [14]
Día 14: Tomas [18]
Día 15: Tomas [19]
Día 16: Tomas [21]
Día 17: Tomas [22]
Día 18: Tomas [24]
Día 19: Tomas [25]
Día 20: Tomas [27]
Día 21: Tomas [28]
Día 22: Tomas [29]
